# Introduction  
このコンペティションは、実際に株式会社日本取引所グループ(Japan Exchange Group, Inc.; JPX)で売買されている株式の銘柄について、将来のリターンが高いものを予測するものとなっています。  
そこでこのノートブックでは、Kaggle初学者では見慣れない`jpx_tokyo_market_prediction`の扱い方と、トレーニングデータにある関連データの取り出し方について取り組みます。  

# Table of Contents  
- [Explanation of data](#Explanation-of-data)
- [jpx_tokyo_market_prediction](#jpx_tokyo_market_prediction)
- [Create models and submit data](#Create-models-and-submit-data)

# Explanation of data

## Loading Modules
まず、必要なモジュールをロードします。  
今回は、pandasを用いてデータのロードを行います。

In [ ]:
import numpy as np
import pandas as pd

## Check the data
pandasの`read_csv`を使い*`stock_price.csv`*を読み込みます。

In [ ]:
stock_price_df = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")

このデータの形(行数・列数)と、中身を確認します。

In [ ]:
print('(行数, 列数) =', stock_price_df.shape)
stock_price_df.tail()

*`stock_price.csv`*に入っていたデータは以下の通りでした。  
*  `SecuritiesCode` ... 証券コード(株式別につけられた番号)
*   `Open` ... 始値(その日の初め(午前9時)の1株当たりの値段)
*  `High` ... 高値(その日の最高値)
*  `Low` ... 低値
*  `Colse` ... 終値
*  `Volume` ... 出来高(１日に売買が成立した株数)
*  `AdjustmentFactor` ... 分割・併合時の理論株価・出来高の算出に使用
*  `ExpectedDividend` ... 権利落ち日の配当予想値
*  `SupercisionFlag` ... 監理銘柄、上場廃止銘柄のフラグ
*  `Target` ... 修正終値の変化率(翌日から翌々日)  
  
今回のコンペでは、他にも多くのデータが用意されていますが、ここでは*`stock_price.csv`*の情報のみを使って実装していきます。

# jpx_tokyo_market_prediction
次に、jpx_tokyo_market_predictionというAPIの使い方を確認します。  
まず、他のモジュールと同様importします。  
※`jpx_tokyo_market_prediction`は一度のみ実行できるので、ここではMarkdown内にイメージを書きます。

***
```python
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()
```


`make_env()`を実行して環境を作成、`iter_test()`を実行してオブジェクトを作成しました。  
下のように、typeをみると`iter_test`はgeneratorなので、for文で1つずつ呼び出せるオブジェクトであることが確認できます。  
***
```python  
print(type(iter_test))
```
[出力]
```
<class 'generator'>
```

for文を回し、以下のように動作確認する。  
***
```Python
count = 0
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    print(prices.head())
    env.predict(sample_prediction)
    count += 1
    break
```
[出力]
```
This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
         Date          RowId  SecuritiesCode    Open    High     Low   Close  \
0  2021-12-06  20211206_1301            1301  2982.0  2982.0  2965.0  2971.0   
1  2021-12-06  20211206_1332            1332   592.0   599.0   588.0   589.0   
2  2021-12-06  20211206_1333            1333  2368.0  2388.0  2360.0  2377.0   
3  2021-12-06  20211206_1375            1375  1230.0  1239.0  1224.0  1224.0   
4  2021-12-06  20211206_1376            1376  1339.0  1372.0  1339.0  1351.0   

    Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag  
0     8900               1.0               NaN            False  
1  1360800               1.0               NaN            False  
2   125900               1.0               NaN            False  
3    81100               1.0               NaN            False  
4     6200               1.0               NaN            False  
```

それぞれの変数名は以下の通りです。
*  `price` ... 対象日の各銘柄のデータ。stock_price.csvのTargetを抜いた情報と同じ。　　
*  `options` ... 対象日のoptions.csvと同じ情報。
*  `finacials` ... 対象日のfinacials.csvと同じ情報。
*  `trades` ... 対象日のtrades.csvと同じ情報。
*  `secondary_prices` ... 対象日のsecandary_stock_price.csvのTargetを抜いた情報と同じ。
*  `sample_prediction` ... 対象日のsample_prediction.csvのデータ。

このように、`jpx_tokyo_market_prediction`を用いて対象となる日付の2000銘柄を1日ずつ呼び出して、作成したモデルで予測し、env.predictで提出データを作成すれば、スコアが出せます。

# Create models and submit data
ここでは、`stock_price.csv`を用いて簡単な学習モデル作成し、提出まで実装します。

## Create Model(LSTM)
**LSTM(Long Short Term Memory)**というモデルを使用します。  
LSTMは系列データに用いるRNNの中の1つで、長期的な依存関係を学習することのできるモデルとなっています。  
Pytorchを用いて、LSTMを実装していきます。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTM(nn.Module):
    def __init__(self, input_size=8, sequence_num=31, lstm_dim=128,
                 num_layers=2, output_size=1):
        super().__init__()
        
        self.lstm = nn.LSTM(input_size, lstm_dim, num_layers, batch_first=True, bidirectional=True)
        self.linear1 = nn.Linear(lstm_dim*sequence_num*2, 1)
        self.bn1 = nn.BatchNorm1d(lstm_dim*sequence_num*2)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        x = lstm_out.reshape(lstm_out.shape[0], -1)
        x = self.linear1(self.bn1(x))
        return x

## Create Dataset
各銘柄ごとに取り出せるデータセットを作成する

まず、stock_price_dfのNanを0に、boolをint型に、*`Date`*をdatetime型に変換する。

In [ ]:
stock_price_df['ExpectedDividend'] = stock_price_df['ExpectedDividend'].fillna(0)
stock_price_df['SupervisionFlag'] = stock_price_df['SupervisionFlag'].map({True: 1, False: 0})
stock_price_df['Date'] = pd.to_datetime(stock_price_df['Date'])
stock_price_df.info()

いくつか、欠損値が含まれていたものがあったので、削除

In [ ]:
stock_price_df = stock_price_df.dropna(how='any')
# 欠損情報確認
stock_price_df_na = (stock_price_df.isnull().sum() / len(stock_price_df)) * 100
stock_price_df_na = stock_price_df_na.drop(stock_price_df_na[stock_price_df_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :stock_price_df_na})
missing_data.head(22)

sklearnのStandardScalerを使って、今回使う特徴量(RowIdとDateとSecuritiesCode以外)を標準化する

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag']
stock_price_df[columns] = stdsc.fit_transform(stock_price_df[columns])
stock_price_df.head()

銘柄ごとにデータを辞書型で保存し、各銘柄ごとに呼び出せるように保存する。

In [ ]:
dataset_dict = {}
for sc in stock_price_df['SecuritiesCode'].unique():
    dataset_dict[str(sc)] = stock_price_df[stock_price_df['SecuritiesCode'] == sc].values[:, 3:].astype(np.float32)
print(dataset_dict['1301'].shape)

Pytorchのデータローダーを使い、ミニバッチごとにデータを呼び出せるようにする。

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, sequence_num=31, y=None, mode='train'):
        self.data = X
        self.teacher = y
        self.sequence_num = sequence_num
        self.mode = mode
    def __len__(self):
        return len(self.teacher)

    def __getitem__(self, idx):
        out_data = self.data[idx]
        if self.mode == 'train':
            out_label =  self.teacher[idx[-1]]
            return out_data, out_label
        else:
            return out_data
def create_dataloader(dataset, dataset_num, sequence_num=31, input_size=8, batch_size=32, shuffle=False):
    sampler = np.array([list(range(i, i+sequence_num)) for i in range(dataset_num-sequence_num+1)])
    if shuffle == True:
        np.random.shuffle(sampler)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size, sampler=sampler)
    return dataloader
#### Check operation ####
X_check, y_check = dataset_dict['1301'][:, :-1], dataset_dict['1301'][:, -1]
dataset_check = MyDataset(X_check, y=y_check, sequence_num=31, mode='train')
dataloader_check = create_dataloader(dataset_check, X_check.shape[0], sequence_num=31, input_size=8, batch_size=32, shuffle=False)
for b, tup in enumerate(dataloader_check):
    print('---------')
    print(tup[0].shape, tup[1].shape)
    break

## Training
各銘柄ごとに、データセットを作成→モデルを学習を繰り返して、LSTMのトレーニングを行っていく。

In [ ]:
from tqdm import tqdm
epochs = 10
batch_size = 512
# Check wheter GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Model Instantiation
model = LSTM(input_size=8, sequence_num=31, lstm_dim=128, num_layers=2, output_size=1)
model.to(device)
model.train()
# setting optimizer
lr = 0.0001
weight_decay = 1.0e-05
optimizer = torch.optim.Adagrad(model.parameters(), lr=lr, weight_decay=weight_decay)
# setting criterion
criterion = nn.MSELoss()
# set iteration counter
iteration = 0
# 
log_train = [[0], [np.inf]]
for epoch in range(epochs):
    epoch_loss = 0.0
    for sc in tqdm(stock_price_df['SecuritiesCode'].unique()):
        X, y = dataset_dict[str(sc)][:, :-1], dataset_dict[str(sc)][:, -1]
        dataset = MyDataset(X, y=y, sequence_num=31, mode='train')
        dataloader = create_dataloader(dataset, X.shape[0], sequence_num=31, input_size=8, batch_size=batch_size, shuffle=True)
        for data, targets in dataloader:
            data, targets = data.to(device), targets.to(device)
            
            optimizer.zero_grad()
            
            data = data.to(torch.float32)
            output = model.forward(data)
            targets = targets.to(torch.float32)
            
            loss = criterion(output.view(1,-1)[0], targets)
            
            loss.backward()
            
            optimizer.step()
            
            epoch_loss += loss.item()
            
            iteration += 1
    epoch_loss /= iteration
    print('epoch_loss={}'.format(epoch_loss))
    log_train[0].append(iteration)
    log_train[1].append(epoch_loss)

学習状況を見るために、損失関数の現象具合を確認する。  
　→学習できている

In [ ]:
import matplotlib.pyplot as plt
plt.plot(log_train[0][1:], log_train[1][1:])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

## Prediction
学習させたモデルを使用して、提出データの予測を行っていく。  
`DataFrame` → `Ndarray` → `tensor`とデータを変換して予測する。

In [ ]:
from datetime import datetime
def predict(model, X_df, sequence=31):
    pred_df = X_df[['Date', 'SecuritiesCode']]
    # groupbyでグループ化して1つずつ取り出す
    code_group = X_df.groupby('SecuritiesCode')
    X_all = np.array([])
    for sc, group in code_group:
        # 対象データを標準化
        group_std = stdsc.transform(group[columns])
        # 対象データの過去データを呼び出す
        X = dataset_dict[str(sc)][-1*(sequence-1):, :-1]
        # 結合
        X = np.vstack((X, group_std))
        X_all = np.append(X_all, X)
    X_all = X_all.reshape(-1, sequence, X.shape[1])
    y_pred = np.array([])
    for it in range(X_all.shape[0]//512+1):
        data = X_all[it*512:(it+1)*512]
        data = torch.from_numpy(data.astype(np.float32)).clone()
        data = data.to(torch.float32)
        data = data.to(device)
        output = model.forward(data)
        output = output.view(1, -1)
        output = output.to('cpu').detach().numpy().copy()
        y_pred = np.append(y_pred, output[0])
    pred_df['target'] = y_pred
    pred_df['Rank'] = pred_df["target"].rank(ascending=False,method="first") -1
    pred_df['Rank'] = pred_df['Rank'].astype(int)
    pred_df = pred_df.drop('target', axis=1)
    return pred_df
test_X_df = stock_price_df[stock_price_df['Date'] == datetime(2021, 12, 3)].drop('Target', axis=1)
y_pred = predict(model, test_X_df)
print(y_pred.shape)
print(y_pred)

# submission

`jpx_tokyo_market_prediction`から、提出データの作成を実施する。

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [ ]:
count = 0
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    prices = prices.fillna(0)
    prices['SupervisionFlag'] = prices['SupervisionFlag'].map({True: 1, False: 0})
    prices['Date'] = pd.to_datetime(prices['Date'])
    pred_df = predict(model, prices)
    print(pred_df)
    env.predict(pred_df)
    count += 1

In [ ]:
pred_df